In [1]:
import numpy as np
import shutil
import pandas as pd
from IPython.display import display
from tqdm import tqdm
from os import walk
import os
import gc

pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
kb_name = "ndsl"
f = open(f'data/{kb_name}/instance_0820.nt', 'r')
trainset = list(map(lambda x : x.split("\t"), f.read().splitlines()))
f.close()

train_df = pd.DataFrame(trainset, columns=['s', 'p', 'o'])
train_df['s'] = train_df['s'].str.replace(' ','_')
train_df['o'] = train_df['o'].str.replace(' ','_')

# pra_path = f'data/{kb_name}/path'
pra_path = f'../pra/Path-ranking-algorithm/{kb_name}'
relation_list = []
for (dirpath, dirnames, filenames) in walk(pra_path):
    relation_list.extend(filenames)

In [8]:
stats = []
totalStats = []
rel_len = len(relation_list)

output_path = f'data/{kb_name}/sentences'

if not os.path.exists(output_path):
    os.mkdir(output_path)

for i, relation in enumerate(relation_list):
    print(f'Relation : {relation}\nCount : {i+1} / {rel_len}')
    
    relDir = relation[:-4]
    
    targetRelation = relDir.replace("-", "/")
    
    target_df =  train_df[train_df.p == targetRelation]
    targetTriple_num = len(target_df)
#     print(targetTriple_num)

    target_df = target_df[['s', 'o']]
    target_df.columns = ['head', 'tail']
    target_df['joinKey'] = target_df['head']

    totalSentence_dir = f'{output_path}/{relDir}'
    if not os.path.exists(totalSentence_dir):
        os.mkdir(totalSentence_dir)
    else:
        shutil.rmtree(totalSentence_dir)
        os.mkdir(totalSentence_dir)


#     relationList = []

    f = open(f'{pra_path}/{relation}', 'r')
    pathList = f.read().splitlines()
#     type processing
    for p in pathList:
        if 'type' in p:
             pathList.remove(p)
    
    
    totalSentence_df = pd.DataFrame()
    target_df =  train_df[train_df.p == targetRelation]
    target_df = target_df[['s']]
    target_df.columns = ['joinKey']
    targetTriple_num = len(target_df)
    
    for pathNum, path in enumerate(pathList[:10]):
        print(f'Relation Count : {i+1} / {rel_len}     Path Count : {pathNum+1} / {len(pathList)}')
        path = path.split(' ')
#         print(f'Path : {path}')
        merged_df = target_df

        last = False
        for rNum, r in enumerate(path):
            if rNum+1 == len(path):
                last =True
            inv = False
            
            if r.startswith("_"):
                r = r[1:]
                inv = True
                
            curR_df = train_df[(train_df.p == r)]
            if rNum == 0:
                if inv:
                    curR_df.columns=["o","p", "joinKey"]
                    merged_df = pd.merge(merged_df, curR_df, on="joinKey")
                    merged_df["sentence"] = (merged_df["joinKey"]+"\tinv_"+merged_df["p"]).replace("\"", "")
                    merged_df = merged_df[["joinKey", "o", "sentence"]]
                else:
                    curR_df.columns=["joinKey","p", "o"]
                    merged_df = pd.merge(merged_df, curR_df, on="joinKey")
                    merged_df["sentence"] = (merged_df["joinKey"]+"\t"+merged_df["p"]).replace("\"", "")
                    merged_df = merged_df[["joinKey", "o","sentence"]]
                
                merged_df.columns=["init_s", "joinKey","sentence"]
                continue
                

            if inv:
                curR_df.columns=["o","p", "joinKey"]
                merged_df = pd.merge(merged_df, curR_df, on="joinKey")
                if last:
                    merged_df["sentence"] = (merged_df["sentence"]+"\tinv_"+merged_df["p"]+"\t"+merged_df["o"]).replace("\"", "")
                else:
                    merged_df["sentence"] = (merged_df["sentence"]+"\tinv_"+merged_df["p"]).replace("\"", "")
                merged_df = merged_df[["init_s", "o", "sentence"]]
            else:
                curR_df.columns=["joinKey","p", "o"]
                merged_df = pd.merge(merged_df, curR_df, on="joinKey")
                if last:
                    merged_df["sentence"] = (merged_df["sentence"]+"\t"+merged_df["p"]+"\t"+merged_df["o"]).replace("\"", "")
                else:
                    merged_df["sentence"] = (merged_df["sentence"]+"\t"+merged_df["p"]).replace("\"", "")
                merged_df = merged_df[["init_s", "o","sentence"]]
            
            merged_df = merged_df.drop_duplicates()
            merged_df.columns=["init_s", "joinKey","sentence"]
        merged_df.columns=["init_s", "last_object", "sentence"]

        sentence_df = merged_df["sentence"]
        totalSentence_df = pd.concat([totalSentence_df, sentence_df])
        
    totalSentence_dir = f'{output_path}/{relDir}'
    if not os.path.exists(totalSentence_dir):
        os.mkdir(totalSentence_dir)
    else:
        shutil.rmtree(totalSentence_dir)
        os.mkdir(totalSentence_dir)
    
    totalSentence_df.to_csv(f'{output_path}/{relDir}/totalSentences.txt', index=False,header=False)
    totalSentence_num = len(totalSentence_df)
    f = open(f'{output_path}/{relDir}/stats.txt', 'w')
    f.write(str((relDir, targetTriple_num, len(pathList), totalSentence_num, round(totalSentence_num/targetTriple_num, 1))))
    f.close()
    
    totalStats.append((relDir, targetTriple_num, len(pathList), totalSentence_num, round(totalSentence_num/targetTriple_num, 1)))

    #totalStats : relation, #Triple, #Path, #Sentence, #AvgSentence
f = open(f'{output_path}/totalStats.txt', 'w')
for stat in totalStats:
    f.write(str(stat)+"\n")
f.close()

Relation : create.txt
Count : 1 / 14
Relation Count : 1 / 14     Path Count : 1 / 3
Relation Count : 1 / 14     Path Count : 2 / 3
Relation Count : 1 / 14     Path Count : 3 / 3
Relation : hasUrl.txt
Count : 2 / 14
Relation Count : 2 / 14     Path Count : 1 / 9
Relation Count : 2 / 14     Path Count : 2 / 9
Relation Count : 2 / 14     Path Count : 3 / 9
Relation Count : 2 / 14     Path Count : 4 / 9
Relation Count : 2 / 14     Path Count : 5 / 9
Relation Count : 2 / 14     Path Count : 6 / 9
Relation Count : 2 / 14     Path Count : 7 / 9
Relation Count : 2 / 14     Path Count : 8 / 9
Relation Count : 2 / 14     Path Count : 9 / 9
Relation : publishedDate.txt
Count : 3 / 14
Relation Count : 3 / 14     Path Count : 1 / 10
Relation Count : 3 / 14     Path Count : 2 / 10
Relation Count : 3 / 14     Path Count : 3 / 10
Relation Count : 3 / 14     Path Count : 4 / 10
Relation Count : 3 / 14     Path Count : 5 / 10
Relation Count : 3 / 14     Path Count : 6 / 10
Relation Count : 3 / 14     Pa